## Importing libraries

In [ ]:
import json
import os

from dotenv import load_dotenv

from genai import Credentials, Client
from genai.schema import TextGenerationParameters, TextGenerationReturnOptions, ModerationHAP, ModerationParameters

## Loading credentials

In [ ]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint)

## Instantiating the GENAI Proxy Object and Client Object

In [ ]:
params = TextGenerationParameters(
    decoding_method="greedy",
    repetition_penalty=1.2,
    min_new_tokens=10,
    max_new_tokens=512,
    return_options=TextGenerationReturnOptions(
        input_text=True,
    )
)
mods = ModerationParameters(
    hap=ModerationHAP(input=True, output=True, threshold=0.75)
)

model = "meta-llama/llama-2-70b"
client = Client(credentials=creds)

## Importing input files

In [ ]:
use_case = 'property-extraction'

In [ ]:
with open(os.path.join(os.getcwd(), f"use-cases/{use_case}/training_examples.json"), "r") as examples_file:
    training_examples = eval(examples_file.read())

In [ ]:
with open(os.path.join(os.getcwd(), f"use-cases/{use_case}/instruction.json"), "r") as instruction_file:
    instruction = json.load(instruction_file)

In [ ]:
with open(os.path.join(os.getcwd(), f"use-cases/{use_case}/test_examples.json"), "r") as tests_file:
    tests = eval(tests_file.read())

## Building prompt template

In [ ]:
pt = """
{{instruction}}

{{examples}}

Input: {{inputs}}
"""

prompt_response = client.prompt.create(
    model_id=model,
    name="property-extraction",
    input=pt,
    parameters=params,
)
prompt_id = prompt_response.result.id

In [ ]:

list_of_inputs = [inputs['input'] for inputs in training_examples]
list_of_outputs = [json.dumps(outputs['output']) for outputs in training_examples]

i=0
example_text = ""
while i < len(list_of_inputs):
    example_text += f"\nInput: {list_of_inputs[i]}\nOutput: {list_of_outputs[i]}\n"
    i += 1

In [ ]:
list_of_test_inputs = [test['input'] for test in tests]

prompt_data = {
    "instruction": instruction['instruction'],
    "examples": example_text,
    "inputs": list_of_test_inputs,
}

## Rendering prompt and generating response

In [ ]:
for response in client.text.generation.create(
    prompt_id=prompt_id,
    data=prompt_data,
    parameters=params,
    moderations=mods
):
    result = response.results[0]
    print(f"Input Text: {result.input_text}")
    print(f"Generated Text:\n{result.generated_text}")
    print("")

## Expected response

In [ ]:
for test in tests:
    print(f"Output: {json.dumps(test['output'])}\n")